<a href="https://colab.research.google.com/github/tejaslinge/Technical-Analysis-using-Python/blob/main/Stoch%2BStochRSI-Bot-Back-Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alpaca-trade-api

# Dependencies

In [2]:
#### config_params.py

import json
config = json.loads(open('ConfigFile.txt', 'r').read()) # Config File with All Parameters
# setting up parameters
# Order Params
global max_trades
global trade_capital_percent
global stop_loss
global trailing_stop
investment_amount = config["investment_amount"]
max_trades = config["max_trades_active"]
trade_capital_percent = config["trade_capital_percent"]
stop_loss = config["stop_loss"]
trailing_stop = config["trailing_stop"]
limit_price = config["limit_price"]
lookback_period = config["candle_lookback_period"]
activate_trailing_stop_loss_at = config["activate_trailing_stop_loss_at"]

# Data Collection Params
global start_date
global end_date
global timeframe
start_date = config["start_date"]
start_date = int(start_date.split()[0])
# start_date = int(start_date)

end_date = config["end_date"]
timeframe = config["timeframe"]
exchange = config["exchange"]

# Technical Indicator (TI) Params
ema = config['indicators']['EMA']
stoch_rsi = config['indicators']['stochRSI']
stoch = config['indicators']['stoch']
ema_params = config['indicators']['EMA_params'] # EMA_Params: Period, Source (Close)
stoch_params = config['indicators']['stoch_params'] # Stoch_Params: K_Length, Lower Band, smooth_D, smooth_K
stochRSI_params = config['indicators']['stochRSI_params'] # StochRSI_Params: K, D, Lower Band, RSI Length, Source (Close)

# ema_period = ema_params['period']
ema_smoothing = ema_params['smoothing']
# ema_fast = ema_params['fast_period']
ema_slow = ema_params['slow_period']

stoch_upper_band = stoch_params['upper_band']
stoch_lower_band= stoch_params['lower_band']
stoch_klength = stoch_params['K_Length']
stoch_smoothk = stoch_params['smooth_K']
stoch_smoothd = stoch_params['smooth_D']

stochRSI_length = stochRSI_params['rsi_length']
stochRSI_rsi_length = stochRSI_params['rsi_length']
stochRSI_upper_band = stochRSI_params['upper_band']
stochRSI_lower_band = stochRSI_params['lower_band']
stochRSI_k = stochRSI_params['K']
stochRSI_d = stochRSI_params['D']

In [3]:
### Signals 
import numpy as np
def implement_stoch_strategy(ticker, upper_band = stoch_upper_band, lower_band = stoch_lower_band):    
    buy_price = []
    sell_price = []
    stoch_signal = []
    signal = 0

    prices = ticker['Close']
    k = ticker['Stoch %K']
    d = ticker['Stoch %D']

    for i in range(len(prices)):
        if k[i] < lower_band and d[i] < lower_band and k[i] < d[i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                stoch_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_signal.append(0)
        elif k[i] > upper_band and d[i] > upper_band and k[i] > d[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                stoch_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            stoch_signal.append(0)

    ticker['Stoch Signal'] = stoch_signal
#     print(stoch_signal)
    return ticker

def implement_ema_strategy(prices, period = ema_slow):
    cols = list(prices.columns)
    ema = prices['Close'].ewm(span = period, adjust = False).mean()
    prices["EMA"] = ema
    prices['Signal_EMA'] = np.where(prices['EMA'] > prices['Close'], 1.0, 0.0)
    prices['EMA Signal'] = prices['Signal_EMA'].diff()    
    cols += ["EMA Signal"]
    prices = prices[cols]
    return prices

def implement_stochRSI_strategy(ticker, upper_band = stochRSI_upper_band, lower_band = stochRSI_lower_band):
    buy_price = []
    sell_price = []
    stoch_signal = []
    signal = 0

    prices = ticker['Close']
    k = ticker['StochRSI_K']
    d = ticker['StochRSI_D']

    for i in range(len(prices)):
        if k[i] < lower_band and d[i] < lower_band and k[i] < d[i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                stoch_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_signal.append(0)
        elif k[i] > upper_band and d[i] > upper_band and k[i] > d[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                stoch_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            stoch_signal.append(0)

    ticker['Stoch RSI Signal'] = stoch_signal
    return ticker

In [4]:
#### indicator.py
############################################################
#TECHNICAL INDICATORS

# calculating Stoch RSI (gives the same values as TradingView)
# https://www.tradingview.com/wiki/Stochastic_RSI_(STOCH_RSI) 
def stochRSI(data, k_window = stochRSI_k, d_window = stochRSI_d, window = stochRSI_length, rsi_window = stochRSI_rsi_length):
        
    def computeRSI (data, rsi_window = rsi_window):
        diff = data.diff(1).dropna()        # diff in one field(one day)
        #this preservers dimensions off diff values
        up_chg = 0 * diff
        down_chg = 0 * diff
        # up change is equal to the positive difference, otherwise equal to zero
        up_chg[diff > 0] = diff[ diff>0 ]
        # down change is equal to negative deifference, otherwise equal to zero
        down_chg[diff < 0] = diff[ diff < 0 ]
        # check pandas documentation for ewm
        # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
        # values are related to exponential decay
        # we set com=time_window-1 so we get decay alpha=1/time_window
        up_chg_avg   = up_chg.ewm(com=rsi_window-1 , min_periods=rsi_window).mean()
        down_chg_avg = down_chg.ewm(com=rsi_window-1 , min_periods=rsi_window).mean()
        rs = abs(up_chg_avg/down_chg_avg)
        rsi = 100 - 100/(1+rs)
        return rsi
    
    data_temp = data['Close']
    data_temp = computeRSI(data_temp, rsi_window)
    
    # input to function is one column from df
    # containing closing price or whatever value we want to extract K and D from
    min_val  = data_temp.rolling(window=window, center=False).min()
    max_val = data_temp.rolling(window=window, center=False).max()
    stoch = ( (data_temp - min_val) / (max_val - min_val) ) * 100
    K = stoch.rolling(window=k_window, center=False).mean() 
    #K = stoch
    D = K.rolling(window=d_window, center=False).mean() 
    
    data['StochRSI_K'] = K
    data['StochRSI_D'] = D    
    return data

def stochastic(ticker, period = stoch_klength, smoothK = stoch_smoothk, smoothD = stoch_smoothd): ## verified
    cols = list(ticker.columns)[1:]
    ticker['14-high'] = ticker['High'].rolling(period).max()
    ticker['14-low'] = ticker['Low'].rolling(period).min()
    ticker['Stoch %K'] = ((ticker['Close'] - ticker['14-low'])*100/(ticker['14-high'] - ticker['14-low'])).rolling(smoothK).mean()
    ticker['Stoch %D'] = ticker['Stoch %K'].rolling(smoothD).mean()
    ticker.drop(columns = ['14-low', '14-high'], inplace = True)
    return ticker

def RSI(series, period=14): # ✔︎✔︎ RSI_EWM
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    return 100 - 100 / (1 + rs)

# calculating Stoch RSI 
#  -- Same as the above function but uses EMA, not SMA
def StochRSI_EMA(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.ewm(span=smoothK).mean()
    stochrsi_D = stochrsi_K.ewm(span=smoothD).mean()

    return stochrsi, stochrsi_K, stochrsi_D

# stochRSI(ticker, period=14, smoothK=3, smoothD=3) VERIFIED
# stochastic(ticker, period = 14, smoothK = 3, smoothD = 3) VERIFIED
# StochRSI_EMA(series, period=14, smoothK=3, smoothD=3) VERIFIED
# RSI(series, period=14) VERIFIED
# EMA (ticker, period = 5, smoothing = 2)

############################################################

In [5]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime as dt
from datetime import timedelta
from math import floor

import alpaca_trade_api as alpaca

# Files
# auth = json.loads(open('AUTH/auth.txt', 'r').read()) # API-KEY and API-SECRET
# client = Client(auth["API-KEY"], auth["API-SECRET"])
key = json.loads(open('authAlpaca.txt', 'r').read())
api = alpaca.REST(key['APCA-API-KEY-ID'], key['APCA-API-SECRET-KEY'], base_url=key["BASE-URL"], api_version = 'v2')
tickers = open('Tickers.txt', 'r').read() # Tickers
tickers = tickers.split()

# Function to fetch data
def get_data(ticker, timeframe= timeframe, start_date = int(start_date), exchanges = exchange):
    df = api.get_crypto_bars(ticker, timeframe, (dt.now() - timedelta(days = start_date)).strftime("%Y-%m-%d"), dt.now().strftime("%Y-%m-%d"), exchanges = exchange).df
#     df.drop(columns = 'exchange', inplace = True)
    df.reset_index(inplace = True)
#     df = df.loc[:, : 'close']

    df = df[['timestamp', 'open', 'high', 'low', 'close']]
    df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close']
    # print(df.columns)
    return df

def stochBacktest(ticker):
    df = get_data(ticker)
    df = stochastic(df)
    # buy_price, sell_price, stoch_signal = implement_stoch_strategy(df['Close'], df['Stoch %K'], df['Stoch %D'])
    df = implement_stoch_strategy(df)
    stoch_signal = df['Stoch Signal']
    df['nextDayClose'] = df['Close'].shift(-1)
    buyDates = []

    for i in range(len(stoch_signal)):
        if stoch_signal[i]==1.0:
            buyDates.append(df['Timestamp'][i])
            
    buyDate = None
    buyDateClose = None
    investment_value = investment_amount
    finalInvestmentValue = investment_amount
    number_of_stocks = None
    newStockValueForTrailing = None
    trailing_stop_loss_at = activate_trailing_stop_loss_at
    changeInInvestmentValue = None
#         buys = []
    sells = []
    soldAtLast = True

    for i in range(0,df.shape[0]):
        if buyDates.count(df['Timestamp'][i]) == 1 and buyDate==None and buyDateClose==None:
            buyDate = df['Timestamp'][i]
            buyDateClose = df['Close'][i]
            number_of_stocks = floor(finalInvestmentValue/buyDateClose)
            finalInvestmentValue -= number_of_stocks*buyDateClose
#                 buys.append(finalInvestmentValue)
            newStockValueForTrailing = buyDateClose*((100+trailing_stop_loss_at)/100)
            soldAtLast = False
#                 print(i, "stocks bought", number_of_stocks, "cost", number_of_stocks*buyDateClose, "total", finalInvestmentValue)

        if buyDate!=None and buyDateClose!=None:
            if df['Close'][i]<=((100-stop_loss)/100)*buyDateClose or df['Close'][i]>=((100+limit_price)/100)*buyDateClose:
                ##direct sell
                finalInvestmentValue += number_of_stocks*df['Close'][i]
                sells.append(finalInvestmentValue)
#                     print(i, "direct stocks sold", number_of_stocks, "cost", number_of_stocks*df['Close'][i], "total", finalInvestmentValue)
                buyDate=None
                buyDateClose=None
                soldAtLast = True
            elif df['Close'][i]>=newStockValueForTrailing:
                minStockValueForTrailing = df['Close'][i]*((100-trailing_stop)/100)
                maxStockValueForTrailing = df['Close'][i]*((100+limit_price)/100)
                if df['nextDayClose'][i]>=maxStockValueForTrailing or df['nextDayClose'][i]<minStockValueForTrailing:
    #                 number_of_stocks = floor(finalInvestmentValue/df['Close'][i])
                    finalInvestmentValue += number_of_stocks*df['Close'][i]
                    sells.append(finalInvestmentValue)
#                         print(i, "stocks sold", number_of_stocks, "cost", number_of_stocks*df['Close'][i], "total", finalInvestmentValue)
                    buyDate=None
                    buyDateClose=None
                    soldAtLast = True

#    
    if len(buyDates) != 0:
      startDate = buyDates[1].strftime("%Y-%m-%d %H:%M:%s")
      endDate = buyDates[-1].strftime("%Y-%m-%d %H:%M:%s")
    else:
      startDate = None
      endDate = None
    print('Timeframe: {}, \nStart Date: {}, \nEnd Date: {}, \nStop Loss: {}, \nTrailing Stop Loss activates at: {}, \nTrailing Stop Loss: {}'.format(timeframe, startDate, endDate, stop_loss, activate_trailing_stop_loss_at, trailing_stop))
    if soldAtLast:
        changeInInvestmentValue =  ((finalInvestmentValue - investment_value)/investment_value)*100
        print('Total Investment Remaining: {}'.format(finalInvestmentValue))
        print('Profit gained from the strategy by investing $100k in {} : {}'.format(ticker, finalInvestmentValue-investment_value))
        print('Profit percentage of the strategy : {}%'.format(changeInInvestmentValue))
    else:
        changeInInvestmentValue =  ((sells[-1] - investment_value)/investment_value)*100
        print('Total Investment Remaining: {}'.format(sells[-1]))
        print('Profit gained from the strategy by investing $100k in {} : {}'.format(ticker, sells[-1]-investment_value))
        print('Profit percentage of the strategy : {}%'.format(changeInInvestmentValue))

def EMAbacktest(ticker):
    df = get_data(ticker)
    df = implement_ema_strategy(df)
    df['nextDayClose'] = df['Close'].shift(-1)
    ema_signal = df['EMA Signal']
    buyDates = []

    for i in range(len(ema_signal)):
        if ema_signal[i]==1.0:
            buyDates.append(df['Timestamp'][i])
            
    buyDate = None
    buyDateClose = None
    investment_value = investment_amount
    finalInvestmentValue = investment_amount
    number_of_stocks = None
    newStockValueForTrailing = None
    trailing_stop_loss_at = activate_trailing_stop_loss_at
    changeInInvestmentValue = None
#         buys = []
    sells = []
    soldAtLast = True

    for i in range(0,df.shape[0]):
        if buyDates.count(df['Timestamp'][i]) == 1 and buyDate==None and buyDateClose==None:
            buyDate = df['Timestamp'][i]
            buyDateClose = df['Close'][i]
            number_of_stocks = floor(finalInvestmentValue/buyDateClose)
            finalInvestmentValue -= number_of_stocks*buyDateClose
#                 buys.append(finalInvestmentValue)
            newStockValueForTrailing = buyDateClose*((100+trailing_stop_loss_at)/100)
            soldAtLast = False
#                 print(i, "stocks bought", number_of_stocks, "cost", number_of_stocks*buyDateClose, "total", finalInvestmentValue)

        if buyDate!=None and buyDateClose!=None:
            if df['Close'][i]<=((100-stop_loss)/100)*buyDateClose or df['Close'][i]>=((100+limit_price)/100)*buyDateClose:
                ##direct sell
                finalInvestmentValue += number_of_stocks*df['Close'][i]
                sells.append(finalInvestmentValue)
#                     print(i, "direct stocks sold", number_of_stocks, "cost", number_of_stocks*df['Close'][i], "total", finalInvestmentValue)
                buyDate=None
                buyDateClose=None
                soldAtLast = True
            elif df['Close'][i]>=newStockValueForTrailing:
                minStockValueForTrailing = df['Close'][i]*((100-trailing_stop)/100)
                maxStockValueForTrailing = df['Close'][i]*((100+limit_price)/100)
                if df['nextDayClose'][i]>=maxStockValueForTrailing or df['nextDayClose'][i]<minStockValueForTrailing:
    #                 number_of_stocks = floor(finalInvestmentValue/df['Close'][i])
                    finalInvestmentValue += number_of_stocks*df['Close'][i]
                    sells.append(finalInvestmentValue)
#                         print(i, "stocks sold", number_of_stocks, "cost", number_of_stocks*df['Close'][i], "total", finalInvestmentValue)
                    buyDate=None
                    buyDateClose=None
                    soldAtLast = True

#         print(soldAtLast, sells)
    if len(buyDates) != 0:
      startDate = buyDates[1].strftime("%Y-%m-%d %H:%M:%s")
      endDate = buyDates[-1].strftime("%Y-%m-%d %H:%M:%s")
    else:
      startDate = None
      endDate = None
    print('Timeframe: {}, \nStart Date: {}, \nEnd Date: {}, \nStop Loss: {}, \nTrailing Stop Loss activates at: {}, \nTrailing Stop Loss: {}'.format(timeframe, startDate, endDate, stop_loss, activate_trailing_stop_loss_at, trailing_stop))
    if soldAtLast:
        changeInInvestmentValue =  ((finalInvestmentValue - investment_value)/investment_value)*100
        print('Total Investment Remaining: {}'.format(finalInvestmentValue))
        print('Profit gained from the strategy by investing $100k in {} : {}'.format(ticker, finalInvestmentValue-investment_value))
        print('Profit percentage of the strategy : {}%'.format(changeInInvestmentValue))
    else:
        changeInInvestmentValue =  ((sells[-1] - investment_value)/investment_value)*100
        print('Total Investment Remaining: {}'.format(sells[-1]))
        print('Profit gained from the strategy by investing $100k in {} : {}'.format(ticker, sells[-1]-investment_value))
        print('Profit percentage of the strategy : {}%'.format(changeInInvestmentValue))
    
def stochRSIBacktest(ticker):
    df = get_data(ticker)
    df = stochRSI(df)
    df = implement_stochRSI_strategy(df)
    stochRSI_signal = df['Stoch RSI Signal']
    df['nextDayClose'] = df['Close'].shift(-1)
    buyDates = []

    for i in range(len(stochRSI_signal)):
        if stochRSI_signal[i]==1.0:
            buyDates.append(df['Timestamp'][i])
            
    buyDate = None
    buyDateClose = None
    investment_value = investment_amount
    finalInvestmentValue = investment_amount
    number_of_stocks = None
    newStockValueForTrailing = None
    trailing_stop_loss_at = activate_trailing_stop_loss_at
    changeInInvestmentValue = None
#         buys = []
    sells = []
    soldAtLast = True

    for i in range(0,df.shape[0]):
        if buyDates.count(df['Timestamp'][i]) == 1 and buyDate==None and buyDateClose==None:
            buyDate = df['Timestamp'][i]
            buyDateClose = df['Close'][i]
            number_of_stocks = floor(finalInvestmentValue/buyDateClose)
            finalInvestmentValue -= number_of_stocks*buyDateClose
#                 buys.append(finalInvestmentValue)
            newStockValueForTrailing = buyDateClose*((100+trailing_stop_loss_at)/100)
            soldAtLast = False
#                 print(i, "stocks bought", number_of_stocks, "cost", number_of_stocks*buyDateClose, "total", finalInvestmentValue)

        if buyDate!=None and buyDateClose!=None:
            if df['Close'][i]<=((100-stop_loss)/100)*buyDateClose or df['Close'][i]>=((100+limit_price)/100)*buyDateClose:
                ##direct sell
                finalInvestmentValue += number_of_stocks*df['Close'][i]
                sells.append(finalInvestmentValue)
#                     print(i, "direct stocks sold", number_of_stocks, "cost", number_of_stocks*df['Close'][i], "total", finalInvestmentValue)
                buyDate=None
                buyDateClose=None
                soldAtLast = True
            elif df['Close'][i]>=newStockValueForTrailing:
                minStockValueForTrailing = df['Close'][i]*((100-trailing_stop)/100)
                maxStockValueForTrailing = df['Close'][i]*((100+limit_price)/100)
                if df['nextDayClose'][i]>=maxStockValueForTrailing or df['nextDayClose'][i]<minStockValueForTrailing:
    #                 number_of_stocks = floor(finalInvestmentValue/df['Close'][i])
                    finalInvestmentValue += number_of_stocks*df['Close'][i]
                    sells.append(finalInvestmentValue)
#                         print(i, "stocks sold", number_of_stocks, "cost", number_of_stocks*df['Close'][i], "total", finalInvestmentValue)
                    buyDate=None
                    buyDateClose=None
                    soldAtLast = True

#         print(soldAtLast, sells)
    
    if len(buyDates) != 0:
      startDate = buyDates[1].strftime("%Y-%m-%d %H:%M:%s")
      endDate = buyDates[-1].strftime("%Y-%m-%d %H:%M:%s")
    else:
      startDate = None
      endDate = None
    print('Timeframe: {}, \nStart Date: {}, \nEnd Date: {}, \nStop Loss: {}, \nTrailing Stop Loss activates at: {}, \nTrailing Stop Loss: {}'.format(timeframe, startDate, endDate, stop_loss, activate_trailing_stop_loss_at, trailing_stop))
    if soldAtLast:
        changeInInvestmentValue =  ((finalInvestmentValue - investment_value)/investment_value)*100
        print('Total Investment Remaining: {}'.format(finalInvestmentValue))
        print('Profit gained from the strategy by investing $100k in {} : {}'.format(ticker, finalInvestmentValue-investment_value))
        print('Profit percentage of the strategy : {}%'.format(changeInInvestmentValue))
    else:
        changeInInvestmentValue =  ((sells[-1] - investment_value)/investment_value)*100
        print('Total Investment Remaining: {}'.format(sells[-1]))
        print('Profit gained from the strategy by investing $100k in {} : {}'.format(ticker, sells[-1]-investment_value))
        print('Profit percentage of the strategy : {}%'.format(changeInInvestmentValue))

def stoch_StochRSI_EMA_Backtest(ticker, stoch=False, stoch_RSI=False, ema=False, lookback_period=1):
    if stoch == False and stoch_RSI == False and ema == False:
        print("Atleast one parameter should be TRUE.")
    elif stoch == True and stoch_RSI == False and ema == False:
        stochBacktest(ticker)
    elif stoch == False and stoch_RSI == True and ema == False:
        stochRSIBacktest(ticker)
    elif stoch == False and stoch_RSI == False and ema == True:
        EMAbacktest(ticker)
    else:        
        df = get_data(ticker)
        print("generate signals for stoch")
    #     print(df)
        df = stochastic(df)
        df = implement_stoch_strategy(df)
        print("generate signals for stochRSI")
        df = stochRSI(df)
        df = implement_stochRSI_strategy(df)
        print("generate signals for ema")
        df = implement_ema_strategy(df)
        df['nextDayClose'] = df['Close'].shift(-1)

        stoch_signal_list = list(df['Stoch Signal'])
        stochRSI_signal_list = list(df['Stoch RSI Signal'])
        ema_signal_list = list(df['EMA Signal'])
        
        positions = []
        buyDates = []
        if len(stoch_signal_list)!=len(stochRSI_signal_list) or len(stochRSI_signal_list)!=len(ema_signal_list) or len(stoch_signal_list)!=len(ema_signal_list):
            print("Some Error")
            return

        for i in range(len(stoch_signal_list)):
            if stoch == True and stoch_RSI == True and ema == True:
                positions.append([stoch_signal_list[i], stochRSI_signal_list[i], ema_signal_list[i]])
            elif stoch == True and stoch_RSI == True and ema == False:
                positions.append([stoch_signal_list[i], stochRSI_signal_list[i]])
            elif stoch == False and stoch_RSI == True and ema == True:
                positions.append([stochRSI_signal_list[i], ema_signal_list[i]])
            elif stoch == True and stoch_RSI == False and ema == True:
                positions.append([stoch_signal_list[i], ema_signal_list[i]])
        
#         for i in range(len(positions)-10):
#             p = np.array(positions[i:i+lookback_period])
#             p[p<=0]=0
#             print("n", p)
#             if sum(p[:,0])>=1 and sum(p[:,1])>=1 and sum(p[:,2])>=1: 
#                 print(True)
#         print("Hi", max(df['Time'][9:100]))
    
        if lookback_period == 1:       
            for i in range(len(positions)):
                if stoch == True and stoch_RSI == True and ema == True and positions[i]==[1,1,1.0]:
                    buyDates.append(df['Timestamp'][i])
                elif stoch == True and stoch_RSI == True and ema == False and positions[i]==[1,1]:
                    buyDates.append(df['Timestamp'][i])
                elif stoch == False and stoch_RSI == True and ema == True and positions[i]==[1,1.0]:
                    buyDates.append(df['Timestamp'][i])
                elif stoch == True and stoch_RSI == False and ema == True and positions[i]==[1,1.0]:
                    buyDates.append(df['Timestamp'][i])
        else:
            for i in range(len(positions)-lookback_period):
                p = np.array(positions[i:i+lookback_period])
                p[p<=0]=0
                if stoch == True and stoch_RSI == True and ema == True:
                    if sum(p[:,0])>=1 and sum(p[:,1])>=1 and sum(p[:,2])>=1:
                        buyDates.append(max(df['Timestamp'][i:i+lookback_period]))
                else:
                    if sum(p[:,0])>=1 and sum(p[:,1])>=1:
                        buyDates.append(max(df['Timestamp'][i:i+lookback_period]))

        buyDate = None
        buyDateClose = None
        investment_value = investment_amount
        finalInvestmentValue = investment_amount
        number_of_stocks = None
        newStockValueForTrailing = None
        trailing_stop_loss_at = activate_trailing_stop_loss_at
        changeInInvestmentValue = None
#         buys = []
        sells = []
        soldAtLast = True

        for i in range(0,df.shape[0]):
            if buyDates.count(df['Timestamp'][i]) == 1 and buyDate==None and buyDateClose==None:
                buyDate = df['Timestamp'][i]
                buyDateClose = df['Close'][i]
                number_of_stocks = floor(finalInvestmentValue/buyDateClose)
                finalInvestmentValue -= number_of_stocks*buyDateClose
#                 buys.append(finalInvestmentValue)
                newStockValueForTrailing = buyDateClose*((100+trailing_stop_loss_at)/100)
                soldAtLast = False
#                 print(i, "stocks bought", number_of_stocks, "cost", number_of_stocks*buyDateClose, "total", finalInvestmentValue)

            if buyDate!=None and buyDateClose!=None:
                if df['Close'][i]<=((100-stop_loss)/100)*buyDateClose or df['Close'][i]>=((100+limit_price)/100)*buyDateClose:
                    ##direct sell
                    finalInvestmentValue += number_of_stocks*df['Close'][i]
                    sells.append(finalInvestmentValue)
#                     print(i, "direct stocks sold", number_of_stocks, "cost", number_of_stocks*df['Close'][i], "total", finalInvestmentValue)
                    buyDate=None
                    buyDateClose=None
                    soldAtLast = True
                elif df['Close'][i]>=newStockValueForTrailing:
                    minStockValueForTrailing = df['Close'][i]*((100-trailing_stop)/100)
                    maxStockValueForTrailing = df['Close'][i]*((100+limit_price)/100)
                    if df['nextDayClose'][i]>=maxStockValueForTrailing or df['nextDayClose'][i]<minStockValueForTrailing:
        #                 number_of_stocks = floor(finalInvestmentValue/df['Close'][i])
                        finalInvestmentValue += number_of_stocks*df['Close'][i]
                        sells.append(finalInvestmentValue)
#                         print(i, "stocks sold", number_of_stocks, "cost", number_of_stocks*df['Close'][i], "total", finalInvestmentValue)
                        buyDate=None
                        buyDateClose=None
                        soldAtLast = True
                        
#         print(soldAtLast, sells)
        # Timeframe = {}
        # Start Date = {}
        # End Date = {}
        # Lookback Period = {}
        # Stop Loss = {}
        # Trailing Stop Loss activates at: {}
        # Trailing Stop Loss = {}
        if len(buyDates) != 0:
          startDate = buyDates[1].strftime("%Y-%m-%d %H:%M:%s")
          endDate = buyDates[-1].strftime("%Y-%m-%d %H:%M:%s")
        else:
          startDate = None
          endDate = None

        print('Parameters for backtesting, Stoch:{}, StochRSI:{}, EMA:{}'.format(stoch, stoch_RSI, ema))
        print('Timeframe: {}, \nStart Date: {}, \nEnd Date: {}, \nLookback Period: {}, \nStop Loss: {}, \nTrailing Stop Loss activates at: {}, \nTrailing Stop Loss: {}'.format(timeframe, startDate, endDate, lookback_period, stop_loss, activate_trailing_stop_loss_at, trailing_stop))
        if soldAtLast:
            changeInInvestmentValue =  ((finalInvestmentValue - investment_value)/investment_value)*100
            print('Total Investment Remaining: {}'.format(finalInvestmentValue))
            print('Profit gained from the strategy by investing $100k in {} : {}'.format(ticker, finalInvestmentValue-investment_value))
        else:
            changeInInvestmentValue =  ((sells[-1] - investment_value)/investment_value)*100
            print('Total Investment Remaining: {}'.format(sells[-1]))
            print('Profit gained from the strategy by investing $100k in {} : {}'.format(ticker, sells[-1]-investment_value))
        
        print('Profit percentage of the strategy : {}%'.format(changeInInvestmentValue))


# Backtesting

In [7]:
stoch_StochRSI_EMA_Backtest('ETHUSD', stoch= False, stoch_RSI = True, ema= True, lookback_period=1)

generate signals for stoch
generate signals for stochRSI
generate signals for ema
Parameters for backtesting, Stoch:False, StochRSI:True, EMA:True
Timeframe: 1Hour, 
Start Date: 2022-02-08 02:00:1644285600, 
End Date: 2022-02-10 02:00:1644458400, 
Lookback Period: 1, 
Stop Loss: 2, 
Trailing Stop Loss activates at: 1, 
Trailing Stop Loss: 2
Total Investment Remaining: 10770.069999999998
Profit gained from the strategy by investing $100k in ETHUSD : 770.0699999999979
Profit percentage of the strategy : 7.700699999999978%
